In [1]:
import geopandas as gpd
from libpysal import graph

import lonboard
from sidecar import Sidecar

In [2]:
buildings = gpd.read_parquet("/data/uscuni-ulce/processed_data/simplified_buildings/buildings_69333.parquet")

In [3]:
layer = lonboard.PolygonLayer.from_geopandas(buildings, opacity=.3)

/Users/martin/micromamba/envs/stable/lib/python3.12/site-packages/lonboard/_geoarrow/ops/reproject.py:107: UserWarning: Input being reprojected to EPSG:4326 CRS.
Lonboard is only able to render data in EPSG:4326 projection.
  warnings.warn(


Create a Sidecar view (assumes JupyterLab) for more comfortable experience.

In [4]:
sc = Sidecar(title='buildings')

Create a Map object

In [5]:
m = lonboard.Map(layer)

Display map within the sidecar plugin

In [6]:
with sc:
    display(m)